# Pointcloud 3D Tree Modelling

This notebook tries to represent a tree in 3D using the deflt repo.

---------------

In [ ]:
# Uncomment to load the local package rather than the pip-installed version.
# Add project src to path.
import set_path

In [ ]:
# Import modules.
import os
import subprocess
import numpy as np
import open3d as o3d
import logging as log
import networkx as nx
import matplotlib.pyplot as plt
from scipy.spatial import KDTree
from shapely.geometry import Polygon
from scipy.spatial import ConvexHull
from plyfile import PlyData, PlyElement

import utils.math_utils as math_utils
import utils.plot_utils as plot_utils
import utils.las_utils as las_utils

## Datasets

#### AHN Reconstruction

![AHN Reconstruction](../imgs/ahn_reconstruction.png)

In [ ]:
ahn_tree_path = '../datasets/single_selection/single_121913_487434_AHN.las'
las_utils.read_las(ahn_tree_path)

# TODO: python reconstruction implementation

#### Cyclo Reconstruction

![Cyclo Media Reconstruction](../imgs/cyclo_reconstruction.png)

In [ ]:
cyclo_tree_path = '../datasets/single_selection/single_121913_487434_Cyclo.las'
cyclo_pcd = las_utils.to_o3d(las_utils.read_las(cyclo_tree_path))

#### Sonarski Reconstruction

![Sonarski Reconstruction](../imgs/sonarski_reconstruction.png)

In [ ]:
sonarski_tree_path = '../datasets/single_selection/single_121913_487434_Sonarski.las'
las_utils.read_las(sonarski_tree_path)

# TODO: python reconstruction implementation

#### Leaf-Wood Filtering

There are 2 filters.
- `surface_variation_filter()`, filter based on the **surface variation** of a point. 
- `curvature_filter()`, filter based on the **curvature** of a point

In [ ]:
def surface_variation_filter(pcd, radius=0.1, threshold=.2):
    pcd.estimate_covariances(
        search_param=o3d.geometry.KDTreeSearchParamRadius(radius=radius))
    eig_val, _ = np.linalg.eig(np.asarray(pcd.covariances))
    eig_val = np.sort(eig_val, axis=1)

    filter_mask = eig_val[:,0] / eig_val.sum(axis=1) < threshold

    return filter_mask

def curvature_filter(pcd, radius, min1=15, max1=75, min2=0, max2=100, min3=0, max3=30):

    # estimate eigenvalues
    pcd.estimate_covariances(
        search_param=o3d.geometry.KDTreeSearchParamRadius(radius=radius))
    eig_val, _ = np.linalg.eig(np.asarray(pcd.covariances))
    eig_val = np.sort(eig_val, axis=1)
    eig_val[eig_val[:,2]==1] = np.zeros(3)
    L1, L2, L3 = eig_val[:,2], eig_val[:,1], eig_val[:,0]

    # filter L1
    filter_L1 = (L1 > L1.min() + (L1.max()-L1.min()) / 100 * min1) & \
        (L1 < L1.min() + (L1.max()-L1.min()) / 100 * max1)

    # filter L2
    filter_L2 = (L2 > L2.min() + (L2.max()-L2.min()) / 100 * min2) & \
        (L2 < L2.min() + (L2.max()-L2.min()) / 100 * max2)

    # filter L3
    filter_L3 = (L3 > L3.min() + (L3.max()-L3.min()) / 100 * min3) & \
        (L3 < L3.min() + (L3.max()-L3.min()) / 100 * max3)


    filter_mask = filter_L1 & filter_L2 & filter_L3
    return filter_mask

def filter_leaves(pcd, surface=False):

    if surface:
        mask = surface_variation_filter(pcd, .05, .2)
    else:
        mask = curvature_filter(pcd, .1, min1=15, max1=50, max3=30)

    # Visualize result
    wood_pcd = cyclo_pcd.select_by_index(np.where(mask)[0])
    wood_pcd = wood_pcd.paint_uniform_color([.5,.3,0])
    leaf_pcd = cyclo_pcd.select_by_index(np.where(mask)[0], invert=True)
    leaf_pcd = leaf_pcd.paint_uniform_color([.8,1,.8])
    o3d.visualization.draw_geometries([wood_pcd, leaf_pcd])

    return wood_pcd


In [ ]:
pcd = cyclo_pcd
pcd_filtered = filter_leaves(pcd)

#### Reconstruction

In [ ]:
# adtree executable
adtree_exe = '../../AdTree-single/build/bin/AdTree.app/Contents/MacOS/AdTree'

In [ ]:
log.basicConfig(
    level=log.INFO,
    format="%(asctime)s [%(levelname)s] %(message)s",
    handlers=[
        log.FileHandler("debug.log"),
        log.StreamHandler()
    ]
)

def create_graph(vertices, edges):

    graph = nx.DiGraph()

    for i, vertex in enumerate(vertices):
        graph.add_node(i, x=vertex[0],y=vertex[1],z=vertex[2])

    for i,j in edges:
        graph.add_edge(j, i)

    return graph

def _run_command(command):
    log.info("Command: {}".format(command))
    result = subprocess.run(command, capture_output=True)
    if result.stderr:
        raise subprocess.CalledProcessError(
                returncode = result.returncode,
                cmd = result.args,
                stderr = result.stderr
                )
    return result

def reconstruct_tree(pcd, adtree_exe):
    '''
    Reconstructs the branches of a point coud tree.

    pcd : open3d.geometry.PointCloud
        input tree point cloud

    adtree : str
        path to adtree executable.

    Returns:
    ----------
    PlyData file of the reconstructed input point cloud tree.
    '''

    skeleton_graph = nx.DiGraph()
    vertices, edges = np.array([]), np.array([])

    # create input file system
    tmp_folder = './tmp'
    in_file = os.path.join(tmp_folder, 'tree.xyz')
    out_file = os.path.join(tmp_folder, 'tree_skeleton.ply')
    if not os.path.exists(tmp_folder):
        os.mkdir(tmp_folder)

    try:    
        log.info("Reconstructing...") 
        o3d.io.write_point_cloud(in_file, pcd) # write input file
        result = _run_command([adtree_exe, in_file, out_file]) # run reconstruction
        plydata = PlyData.read(out_file) # read output

        # Convert skeleton to graph
        log.info("Done. Constructing graph...")
        vertices = np.array([[c for c in p] for p in plydata['vertex'].data])
        vertices, reverse_ = np.unique(vertices, axis=0, return_inverse=True)
        edges = np.array([reverse_[edge[0]] for edge in plydata['edge'].data])
        skeleton_graph = create_graph(vertices, edges)
        log.info("Done. Succesful.")
        
    except subprocess.CalledProcessError as e:
        log.error("Failed reconstructing tree:\n{}".format(e.stderr.decode('utf-8')))
    except Exception as e:
        log.error("Failed:\n{}".format(e))

    # clean filesystem
    if os.path.exists(in_file):
        os.remove(in_file)
    if os.path.exists(out_file):
        os.remove(out_file)
    if os.path.isdir(tmp_folder):
        os.rmdir(tmp_folder)

    return skeleton_graph, vertices, edges

def plot_skeleton(vertices, edges):
    # visulalize
    colors = [[0, 0, 0] for i in range(len(edges))]
    line_set = o3d.geometry.LineSet()
    line_set.points = o3d.utility.Vector3dVector(vertices)
    line_set.lines = o3d.utility.Vector2iVector(edges)
    line_set.colors = o3d.utility.Vector3dVector(colors)
    o3d.visualization.draw_geometries([line_set])

def branch_path(graph, start):

    path = [start]
    while graph.out_degree(path[-1]) == 1:
        for node in graph.successors(path[-1]):
            path.append(node)
            break

    print(path[0], '-->', path[-1], '(nodes:',len(path),')')
    return path


In [ ]:
skeleton, vertices, edges = reconstruct_tree(pcd_filtered, adtree_exe)

In [ ]:
plot_skeleton(vertices, edges)

In [ ]:
# Get Tree Stem

# Get skeleton till first branch
start = np.argmin(vertices[:,2])
stem_path = branch_path(skeleton, start)
skeleton_points = vertices[stem_path]

# Create tree points
tree_points = np.array(pcd.points)
mask_idx = np.where(tree_points[:,2] < skeleton_points[:,2].max())[0]
tree = KDTree(tree_points[mask_idx])

# Filter tree points
stem_points_idx = set()
num_ = int(np.linalg.norm(skeleton_points[1]-skeleton_points[0]) / 0.05)
stem_points = np.linspace(start=skeleton_points[0], stop=skeleton_points[1], num=num_)
for result in tree.query_ball_point(stem_points, .75):
    stem_points_idx.update(result) 
stem_points_idx = mask_idx[list(stem_points_idx)]

# Visualize
stem_pcd = pcd.select_by_index(stem_points_idx)
stem_pcd = stem_pcd.paint_uniform_color([.4,.7,0])
noise_pcd = pcd.select_by_index(stem_points_idx, invert=True)
o3d.visualization.draw_geometries([stem_pcd, noise_pcd])